In [1]:
#import all neccessary libraries
import nltk
import numpy as np
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vishalbarad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vishalbarad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vishalbarad/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
data = pd.read_csv('user_reviews_ratings.csv',encoding = 'unicode_escape')

In [3]:
data.sample(10)

,Reviews,Ratings
739,\n Just bought yesterday using it and loving ...,4.0 out of 5 stars
3236,\n Camera quality very bad\n,1.0 out of 5 stars
1013,"\n User interface is undoubtedly the best, bu...",4.0 out of 5 stars
1343,\n Fingerprint is not up to the Mark. It's ve...,5.0 out of 5 stars
3320,\n Good product\n,5.0 out of 5 stars
3743,\n Good\n,5.0 out of 5 stars
1184,\n Camera quality didn't meet expectations......,3.0 out of 5 stars
1521,\n Fingerprint sensor works fine. Battery lif...,1.0 out of 5 stars
3022,\n Too good\n,5.0 out of 5 stars
219,\n Best mid range phone available in the mark...,4.0 out of 5 stars


In [4]:
#Use lemmatization technique of NLP to convert inflacted word to its base form
lemmatizer = WordNetLemmatizer()

In [5]:
data['Reviews'] = data['Reviews'].astype('str')

In [6]:
corpus=[]
for i in range(len(data)):
  review = re.sub('[^a-zA-Z]',' ',data['Reviews'][i])#remove punctuation and num
  review = review.lower()
  review = review.split()#conver into list
  review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words('english'))]
  review = ' '.join(review)
  corpus.append(review)


In [7]:
data['Reviews'] = corpus

In [8]:
data.head()

,Reviews,Ratings
0,bakwas phone,1.0 out of 5 stars
1,issue resolved mention still mobile charm oneplus,3.0 out of 5 stars
2,best part phone super simple user friendly ope...,5.0 out of 5 stars
3,phone look good feature k phone,1.0 out of 5 stars
4,performance really good lag,5.0 out of 5 stars


In [9]:
# Fetch rating from string
Ratings = []
for i in range(len(data)):
    rating = re.findall('^\d.\d',data['Ratings'][i])
    rating = float(rating[0])
    rating = int(rating)
    Ratings.append(rating)

In [10]:
data['Ratings'] = Ratings

In [11]:
data.head()

,Reviews,Ratings
0,bakwas phone,1
1,issue resolved mention still mobile charm oneplus,3
2,best part phone super simple user friendly ope...,5
3,phone look good feature k phone,1
4,performance really good lag,5


In [12]:
# Divide rating into the "low","average","high" categories as 
# rating>3 is high
# rating=3 is average
# rating<3 is low

data['Desc_Ratings'] = "true"
for rating_index,rating_data in data['Ratings'].iteritems():
    if rating_data>3:
        data['Desc_Ratings'][rating_index] = "high"
    elif rating_data==3:
        data['Desc_Ratings'][rating_index] = "average"
    else:
        data['Desc_Ratings'][rating_index] = "low"

/Users/vishalbarad/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/vishalbarad/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/vishalbarad/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [13]:
data

,Reviews,Ratings,Desc_Ratings
0,bakwas phone,1,low
1,issue resolved mention still mobile charm oneplus,3,average
2,best part phone super simple user friendly ope...,5,high
3,phone look good feature k phone,1,low
4,performance really good lag,5,high
...,...,...,...
4006,good product,5,high
4007,mazing phone worth money,4,high
4008,satisfied,1,low
4009,good,3,average


In [15]:
# perform train-test split on data
x_train,x_test,y_train,y_test=train_test_split(data['Reviews'],data['Desc_Ratings'],test_size=0.2,random_state=0)

In [16]:
# Create pipeline to perform TF-IDF and apply MultinomialNB classifier algo to perform classification
clf = Pipeline([('vectorize',CountVectorizer()),('nb',MultinomialNB())])
clf.fit(x_train,y_train)

Pipeline(steps=[('vectorize', CountVectorizer()), ('nb', MultinomialNB())])

In [17]:
# Predict the score
clf.score(x_test,y_test)

0.7173100871731009

In [22]:
# Predcit the output for particular input
clf.predict(['bakwas phone'])[0]

'low'

In [23]:
clf.predict(['good product'])[0]

'high'

In [24]:
clf.predict(['product is very good, battery life is very long and good, I really love this product'])[0]

'high'

In [25]:
clf.predict(['bad product'])[0]

'low'